# Ubelt's CacheStamp class

The `ubelt.CacheStamp` class is used to mark that a block of code has been run, and it's output has been written to disk. 

You set up a `CacheStamp` by giving it a name and letting it know what files we expect to already exist or that need to be written. Then you, check if the stamp is "expired". If it is, you need to recompute the data you wish to cache and "renew" the stamp. If it is not expired, then you can expect that:

1. The file already exist on disk.
2. The file has not been tampered with since you wrote it.

Running renew records the size, modification time (mtime), and hash (checksum) of each file registered via product. Running expired checks checks that these attributes match with existing files on disk, which gives you the tamperproof guarantee. This mechanism is similar to how Makefiles and other build systems (e.g. CMake, redo) handle detecting when files are modified. (Note that it is possible to disable the hash checks by specifying `hasher=None` while still retaining size and mtime checks, this is useful when hashing files it too expensive).

In [33]:
import ubelt as ub

dpath = ub.Path.appdir('stamp-demo').delete().ensuredir()
fpath1 = dpath / 'large-file1.txt'
fpath2 = dpath / 'large-file2.txt'

stamp = ub.CacheStamp('stamp-name', dpath=dpath, product=[fpath1, fpath2])

# If the stamp is expired, we need to recompute the process
if stamp.expired():
    fpath1.write_text('large-data1')
    fpath2.write_text('large-data2')
    # After the process is complete, renew the stamp
    stamp.renew()

# Next time the code is run, the stamp will not be expired
assert not stamp.expired()

[cacher] ... stamp-name cache miss
[cacher] stamp expired no_cert
[cacher] ... stamp-name cache save


The 1.1.0 implementation of `CacheStamp` also contains other features. For instance, you can set an expiration duration or time for the file to expire. All properties can be updated via the constructor or by setting instance attributes. We can demo the expired property by reusing the above stamp.

In [34]:
import time

# Tell the stamp it will expire 2 seconds, and renew it to set that property.
stamp.expires = 2
stamp.renew()

assert not stamp.expired(), 'should not be expired yet'

# Wait 2 seconds
time.sleep(2.1)

# The stamp is now expired
assert stamp.expired(), 'the stamp should be expired'

[cacher] ... stamp-name cache save
[cacher] stamp expired expired_cert


You can also specify an expected hash prefix for each file, which is useful when you know what file will be produced a-priori (e.g. downloading a known file, in fact the `ubelt.grabdata` mechanism is now implemented with `ubelt.CacheStamp`). It works something like this:

In [35]:
import ubelt as ub

url = 'https://github.com/Kitware/CMake/releases/download/v3.22.5/cmake-3.22.5.tar.gz'
dpath = ub.Path.appdir('stamp-download-demo').delete().ensuredir()

fpath = dpath / 'cmake-3.22.5.tar.gz'
stamp = ub.CacheStamp(
    'download-stamp',
    dpath=dpath, 
    product=fpath,
    hash_prefix='057d3d40d49fe1503edb62735a73de399d90c92c',
)
if stamp.expired():
    ub.download(url, fpath=fpath)
    stamp.renew()

[cacher] ... download-stamp cache miss
[cacher] stamp expired no_cert
 9785396/9785396... rate=19792642.19 Hz, eta=0:00:00, total=0:00:00
[cacher] ... download-stamp cache save


The new features added in 1.1.0 were:

* Supporting expiration time
* Supporting mtime and size checks
* Supporting an expected hash-prefix


https://github.com/Erotemic/ubelt/releases/tag/v1.1.0